<a href="https://colab.research.google.com/github/felix2024/CursosDeHTMl/blob/main/Copia_de_DatosTopicosDeIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Practica de datos**







**¿De que trata este dataset?**

Este conjunto de datos es un registro de monitoreo de calidad del agua en diversos cuerpos superficiales (ríos y presas), principalmente en la región de Sinaloa, México (específicamente en la cuenca de los ríos Culiacán, Tamazula y Humaya).

**¿Cual es la finalidad de esta practica?**


1.   Realizar un analisis del conjunto de datos
2.   Preguntar a la IA 3 métodos diferentes para imputar los datos faltantes e implentar los métodos.


1.   Buscar e implementar una evaluación de las imputaciones realizadas
2.   Normalizar con dos métodos diferentes de normalización, explicar diferencias y uso





**DATASET ANTES**
[Base de datos conagua](https://drive.google.com/file/d/1oXuWvkdxYb855T7N93SDuSiRynstQEKw/view?usp=sharing)

# **Fase 1: Comprensión y Análisis del Conjunto de Datos**



1.   Resumen general


  *   Dimensiones: 6,095 filas y 10 columnas.
  *   Tipos de Variables: - Numéricas: Inicialmente se detectaron pocas, pero tras una inspección, columnas como OD_%, SST, N_TOT, P_TOT y CAUDAL estaban en formato texto debido a caracteres como < o >.

  *   Categóricas: CLAVE SITIO y NOMBRE DEL SITIO.




In [ ]:
import pandas as pd
import numpy as np

# Carga del dataset
df = pd.read_csv('Datos para Imputación.csv')

# Función para limpiar valores especiales (ej. "<5", ">10") y convertir a float
def clean_to_numeric(val):
    if pd.isna(val):
        return np.nan
    val = str(val).strip()
    if not val:
        return np.nan
    # Heurística: Si detecta '<', usa la mitad del límite de detección
    if val.startswith('<'):
        try: return float(val[1:]) / 2
        except: return np.nan
    # Si detecta '>', usa el valor numérico
    if val.startswith('>'):
        try: return float(val[1:])
        except: return np.nan
    try:
        return float(val.replace(',', ''))
    except ValueError:
        return np.nan

# Aplicar limpieza a las columnas identificadas como texto pero que deberían ser numéricas
cols_to_convert = ['OD_%', 'SST', 'N_TOT', 'P_TOT', 'CAUDAL']
for col in cols_to_convert:
    df[col] = df[col].apply(clean_to_numeric)

# Separar columnas para el procesamiento
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

print("Conteo de valores nulos para columnas numéricas después de la limpieza de caracteres especiales:")
print(df[numerical_cols].isnull().sum())

Conteo de valores nulos para columnas numéricas después de la limpieza de caracteres especiales:
CONDUC_CAMPO      56
PH_CAMPO          95
TEMP_AGUA        271
OD_%             247
SST               23
N_TOT              0
P_TOT              0
CAUDAL          3169
dtype: int64




  
2.   Análisis de Valores Faltantes:



In [ ]:
# Calcular el porcentaje de valores nulos para cada columna numérica
nulos_porcentaje = (df[numerical_cols].isnull().sum() / len(df)) * 100

# Ordenar los resultados para una mejor visualización
nulos_porcentaje = nulos_porcentaje.sort_values(ascending=False)

print("Porcentaje de valores nulos por columna numérica:")
print(nulos_porcentaje)

Porcentaje de valores nulos por columna numérica:
CAUDAL          51.993437
TEMP_AGUA        4.446267
OD_%             4.052502
PH_CAMPO         1.558655
CONDUC_CAMPO     0.918786
SST              0.377358
N_TOT            0.000000
P_TOT            0.000000
dtype: float64


Tras limpiar los caracteres especiales (ej. convertir <5 a 2.5), el estado de los nulos es:



*   CAUDAL: 51.99% de nulos (Crítico).

*   ETEMP_AGUA: 4.45% de nulos.
*   OD_%: 4.05% de nulos.


*  PH_CAMPO: 1.56% de nulos.


*   CONDUC_CAMPO: 0.92% de nulos.






# **Fase 2: Diseño de Estrategias de Imputación**

Ahora diseñamos la estrategias de imputacion para generar tres versiones del dataset



1.   Imputación por Mediana (Simple)


*   Concepto: Reemplaza el nulo por el valor central de la columna.
*   Ventaja: No se ve afectada por valores atípicos (outliers).

*   Uso: Cuando los datos faltan de forma aleatoria y la distribución es sesgada.


2.   K-Nearest Neighbors (KNN Imputer):


*   Concepto: Encuentra las $k$ filas más similares a la fila con el nulo y promedia sus valores.
*   Ventaja: Captura relaciones locales entre variables.

*   Uso: Problemas complejos donde las variables dependen unas de otras.






3.   Iterative Imputer (MICE):


*   Concepto: Modela cada variable como una función de las demás en un proceso iterativo.
*   Ventaja: Es el estándar de oro para mantener la estructura multivariada.

*  Uso: Problemas complejos donde las variables dependen unas de otras.











In [ ]:
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

X = df[numerical_cols].copy()

# 1. Método Simple (Mediana)
simple_imp = SimpleImputer(strategy='median')
X_simple = pd.DataFrame(simple_imp.fit_transform(X), columns=numerical_cols)

# 2. Método KNN (K-Nearest Neighbors)
knn_imp = KNNImputer(n_neighbors=5)
X_knn = pd.DataFrame(knn_imp.fit_transform(X), columns=numerical_cols)

# 3. Método Iterativo (MICE)
iter_imp = IterativeImputer(random_state=42)
X_iter = pd.DataFrame(iter_imp.fit_transform(X), columns=numerical_cols)

# **Fase 3: Evaluación de las Imputaciones**

Para decidir cuál es el mejor método, evalué dos criterios:




1.   Preservación de la Media: Qué tanto cambia el promedio original tras la imputación.
2.   Preservación de Correlaciones: Se calculó la diferencia absoluta promedio entre la matriz de correlación original y la imputada.



**¿El porque usar estas metricas?**



1.   Media: es una medida central clave que representa el valor típico de una variable. Si un método de imputación altera significativamente la media original de una columna, significa que está introduciendo un sesgo en los datos. Por ejemplo, si el valor promedio de TEMP_AGUA después de la imputación es mucho más alto o bajo que el original, las conclusiones sobre la temperatura promedio del agua serían incorrectas. Mantener la media asegura que la tendencia central de la variable se conserve.
2.   Correlaciones: Las correlaciones describen las relaciones lineales entre diferentes variables en tu dataset. Muchos modelos estadísticos y de Machine Learning (como la regresión, el análisis de componentes principales, etc.) dependen críticamente de estas relaciones. Si la imputación distorsiona las correlaciones, los modelos construidos sobre los datos imputados podrían extraer conclusiones erróneas o tener un rendimiento deficiente. Preservar las correlaciones asegura que la estructura relacional entre las variables se mantenga intacta.


In [ ]:
# =================================================================
# FASE 3: EVALUACIÓN INTEGRAL DE MÉTODOS DE IMPUTACIÓN
# =================================================================

import pandas as pd
import numpy as np

# 1. ESTABLECER VALORES DE REFERENCIA (DATOS ORIGINALES)
# Usamos el df original antes de la imputación para calcular la "verdad"
medias_originales = df[numerical_cols].mean()
corr_original = df[numerical_cols].corr()

# 2. FUNCIÓN MAESTRA DE EVALUACIÓN
def evaluar_metodo_completo(df_imputado, nombre_metodo):
    """
    Calcula el error en media por columna y la distorsión
    estructural global para un método específico.
    """
    cols = numerical_cols

    # Métricas de Media
    media_imp = df_imputado[cols].mean()
    error_media = (medias_originales - media_imp).abs()

    # Métricas de Estructura (Correlación)
    # Comparamos la matriz de correlación del imputado vs la original
    corr_imp = df_imputado[cols].corr()
    dist_metodo = (corr_imp - corr_original).abs().mean().mean()

    # Crear DataFrame con los resultados del método
    resumen = pd.DataFrame({
        'Método': nombre_metodo,
        'Variable': cols,
        'Media Original': medias_originales.values,
        'Media del Método': media_imp.values,
        'Error en la Media': error_media.values,
        'Dist. Estructural Original': 0.0, # Punto de partida ideal
        'Dist. Estructural Método': dist_metodo # Error estructural acumulado
    })
    return resumen

# 3. EJECUCIÓN DE LA EVALUACIÓN PARA LOS TRES MÉTODOS
# Procesamos los resultados obtenidos en la Fase 2
eval_simple = evaluar_metodo_completo(X_simple, "Simple (Mediana)")
eval_knn = evaluar_metodo_completo(X_knn, "KNN (Vecinos)")
eval_iter = evaluar_metodo_completo(X_iter, "Iterative (MICE)")

# 4. CONSOLIDACIÓN EN TABLA FINAL
tabla_evaluacion_final = pd.concat([eval_simple, eval_knn, eval_iter], ignore_index=True)

# 5. VISUALIZACIÓN DE RESULTADOS
print("--- TABLA DE EVALUACIÓN FINAL: COMPARATIVA DE IMPUTACIONES ---")
# Mostramos la tabla completa
display(tabla_evaluacion_final)

# 6. RESUMEN EJECUTIVO (PROMEDIO DE ERRORES POR MÉTODO)
resumen_ejecutivo = tabla_evaluacion_final.groupby('Método').agg({
    'Error en la Media': 'mean',
    'Dist. Estructural Método': 'first' # Es el mismo para todas las variables del método
}).sort_values('Dist. Estructural Método')

print("\n--- RESUMEN POR MÉTODO (ORDENADO POR MEJOR DESEMPEÑO ESTRUCTURAL) ---")
print(resumen_ejecutivo)

# Guardar la evaluación para registro
tabla_evaluacion_final.to_csv('Evaluacion_Fase3_Completa.csv', index=False)

--- TABLA DE EVALUACIÓN FINAL: COMPARATIVA DE IMPUTACIONES ---


,Método,Variable,Media Original,Media del Método,Error en la Media,Dist. Estructural Original,Dist. Estructural Método
0,Simple (Mediana),CONDUC_CAMPO,18566.362578,18402.420280,163.942298,0.0,0.011682
1,Simple (Mediana),PH_CAMPO,8.110974,8.110803,0.000171,0.0,0.011682
2,Simple (Mediana),TEMP_AGUA,28.619719,28.645520,0.025801,0.0,0.011682
3,Simple (Mediana),OD_%,93.512249,93.609013,0.096764,0.0,0.011682
4,Simple (Mediana),SST,65.142921,64.982005,0.160917,0.0,0.011682
5,Simple (Mediana),N_TOT,2.830067,2.830067,0.000000,0.0,0.011682
6,Simple (Mediana),P_TOT,0.365310,0.365310,0.000000,0.0,0.011682
7,Simple (Mediana),CAUDAL,13378.967220,7370.284698,6008.682522,0.0,0.011682
8,KNN (Vecinos),CONDUC_CAMPO,18566.362578,18623.477415,57.114837,0.0,0.007531
9,KNN (Vecinos),PH_CAMPO,8.110974,8.111068,0.000094,0.0,0.007531



--- RESUMEN POR MÉTODO (ORDENADO POR MEJOR DESEMPEÑO ESTRUCTURAL) ---
                  Error en la Media  Dist. Estructural Método
Método                                                       
KNN (Vecinos)            583.802082                  0.007531
Simple (Mediana)         771.613559                  0.011682
Iterative (MICE)         634.767760                  0.015076


# Fase 4: Normalización de los Datos

Con el dataset imputado por KNN, apliqué dos métodos de normalización:




Min-Max Scaling: Escala los datos al
rango $[0, 1]$.Uso: Ideal para algoritmos de redes neuronales o basados en distancia (como el mismo KNN).





Standardization (Z-score): Escala a media 0 y desviación estándar 1.Uso: Recomendado para modelos lineales o análisis de componentes principales (PCA), ya que asume una distribución normal.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# 1. Min-Max Scaling (Rango 0 a 1)
minmax_scaler = MinMaxScaler()
X_minmax = pd.DataFrame(minmax_scaler.fit_transform(X_best), columns=numerical_cols)

# 2. Estandarización (Z-Score)
std_scaler = StandardScaler()
X_std = pd.DataFrame(std_scaler.fit_transform(X_best), columns=numerical_cols)

# Comparativa de resultados
print("\nEjemplo de normalización (PH_CAMPO):")
comparison = pd.DataFrame({
    'Imputado': X_best['PH_CAMPO'].head(),
    'MinMax': X_minmax['PH_CAMPO'].head(),
    'Standard': X_std['PH_CAMPO'].head()
})
print(comparison)


Ejemplo de normalización (PH_CAMPO):
   Imputado    MinMax  Standard
0      8.20  0.816327  0.192935
1      7.97  0.792857 -0.306041
2      8.02  0.797959 -0.197568
3      8.05  0.801020 -0.132484
4      8.27  0.823469  0.344797


# Fase 5: Integración y Conclusione

**Resultados Finales:**


*   Mejor Método de Imputación: KNN (k=5). Logró recupera
*   Mejor Normalización: Se recomienda la Estandarización (Z-score) para este dataset, dado que variables como CONDUC_CAMPO tienen rangos muy amplios (de 26 a 148,000) y la estandarización maneja mejor estos extremos que el Min-Max.



**Conclusiones**

El dataset ha sido saneado de errores de formato, se han recuperado los valores nulos manteniendo la integridad estadística y se ha entregado un archivo listo para modelos de Machine Learning.

In [ ]:
# Unimos con las columnas categóricas (nombres de sitio, etc.)
df_final = pd.concat([df[categorical_cols].reset_index(drop=True), X_std], axis=1)

# Exportación del resultado final
df_final.to_csv('Dataset_Final_Procesado.csv', index=False)
print("\nProceso completado con éxito. Archivo 'Dataset_Final_Procesado.csv' generado.")


Proceso completado con éxito. Archivo 'Dataset_Final_Procesado.csv' generado.
